<a href="https://colab.research.google.com/github/qsardor/GoogleColabProjects/blob/main/Colab_noVNC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title 🖥️ Colab noVNC Ultimate Edition
# ============================================================================
# PROJECT: Colab noVNC Ultimate Edition
# DESCRIPTION: XFCE4 | Chrome | Terminator | Panic Button | Smart Auto-Detect
# CREATOR: Q.SARDOR
#
# 🔗 MORE PROJECTS: https://github.com/qsardor/GoogleColabProjects
# 📢 TELEGRAM: https://t.me/qsardorblog
# ============================================================================

import os
import subprocess
import time
import sys
import urllib.request
import shutil

# ==========================================
# ⚙️ USER CONFIGURATION
# ==========================================
Mount_Google_Drive = False # @param {type:"boolean"}

# Internal Config
VNC_PASSWORD = "colabuser"
USER_NAME = "colab"
HOME_DIR = f"/home/{USER_NAME}"
SYSTEM_PATH = "/usr/local/bin"

# ==========================================
# 1. SYSTEM INFO & HARDWARE CHECK
# ==========================================
def get_system_info():
    try:
        # OS Version
        os_ver = subprocess.check_output("grep PRETTY_NAME /etc/os-release | cut -d'=' -f2 | tr -d '\"'", shell=True).decode().strip()
        # RAM
        ram = subprocess.check_output("free -h | awk '/Mem:/ {print $2}'", shell=True).decode().strip()
        # Storage
        storage = subprocess.check_output("df -h / | awk 'NR==2 {print $2}'", shell=True).decode().strip()
        return os_ver, ram, storage
    except:
        return "Ubuntu Unknown", "Unknown", "Unknown"

def detect_gpu():
    print("🔍 [1/9] Auto-Detecting Hardware...")
    try:
        subprocess.check_output("nvidia-smi", shell=True)
        print("    ✅ NVIDIA GPU Detected. Switching to GPU Mode.")
        return True
    except subprocess.CalledProcessError:
        print("    ℹ️  No GPU Detected. Switching to CPU Mode.")
        return False

def mount_drive():
    if Mount_Google_Drive:
        print("📂 [2/9] Mounting Google Drive...")
        from google.colab import drive
        drive.mount('/content/drive')
        print("    ✅ Drive Mounted.")
        return "Enabled"
    else:
        print("📂 [2/9] Google Drive Skipped.")
        return "Disabled"

# ==========================================
# 2. INSTALLATION
# ==========================================
def install_system(has_gpu):
    print("📦 [3/9] Installing System & Tools...")

    subprocess.run("apt-get update -qq", shell=True)

    packages = [
        "xfce4",
        "xfce4-terminal",
        "tigervnc-standalone-server",
        "tigervnc-xorg-extension",
        "dbus-x11",
        "terminator",
        "htop",
        "nano",
        "wget",
        "sudo",
        "x11-xserver-utils",
        "npm"
    ]

    if has_gpu:
        packages.extend(["mesa-utils", "vulkan-tools", "libvulkan1"])

    cmd = f"apt-get install -y -qq {' '.join(packages)}"
    subprocess.run(cmd, shell=True)

    # Install Chrome
    print("    ├── Installing Google Chrome...")
    subprocess.run("wget -q https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb", shell=True)
    subprocess.run("apt-get install -y ./google-chrome-stable_current_amd64.deb", shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    subprocess.run("rm google-chrome-stable_current_amd64.deb", shell=True)

    # Install Localtunnel
    subprocess.run("npm install -g localtunnel", shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

    print("    ✅ System Installed.")

def create_user():
    print("👤 [4/9] Creating Non-Root User...")
    subprocess.run(f"useradd -m -s /bin/bash {USER_NAME}", shell=True)
    subprocess.run(f"usermod -aG sudo {USER_NAME}", shell=True)
    subprocess.run(f"echo '{USER_NAME} ALL=(ALL) NOPASSWD:ALL' >> /etc/sudoers", shell=True)
    print(f"    ✅ User '{USER_NAME}' created.")

def setup_novnc():
    print("🌐 [5/9] Setting up noVNC...")
    if not os.path.exists("/opt/noVNC"):
        subprocess.run("git clone https://github.com/novnc/noVNC.git /opt/noVNC", shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        subprocess.run("git clone https://github.com/novnc/websockify /opt/noVNC/utils/websockify", shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        subprocess.run("ln -s /opt/noVNC/vnc.html /opt/noVNC/index.html", shell=True)
    print("    ✅ noVNC Ready.")

# ==========================================
# 3. CONFIGURATION
# ==========================================
def configure_vnc_user():
    print("⚙️ [6/9] Configuring VNC for User...")

    vnc_dir = f"{HOME_DIR}/.vnc"
    os.makedirs(vnc_dir, exist_ok=True)

    passwd_file = os.path.join(vnc_dir, "passwd")
    with open(passwd_file, "wb") as f:
        f.write(subprocess.check_output(["vncpasswd", "-f"], input=VNC_PASSWORD.encode()))
    os.chmod(passwd_file, 0o600)

    xstartup_content = """#!/bin/bash
xrdb $HOME/.Xresources
export USER=colab
export HOME=/home/colab
unset SESSION_MANAGER
unset DBUS_SESSION_BUS_ADDRESS

# PERFORMANCE TWEAKS
xfconf-query -c xfwm4 -p /general/use_compositing -s false --create -t bool
xfconf-query -c xfce4-desktop -p /backdrop/screen0/monitor0/workspace0/last-image -s "" --create -t string
xfconf-query -c xfce4-desktop -p /backdrop/screen0/monitor0/workspace0/color-style -s 0 --create -t int

dbus-launch --exit-with-session /usr/bin/startxfce4
"""
    xstartup_path = os.path.join(vnc_dir, "xstartup")
    with open(xstartup_path, "w") as f:
        f.write(xstartup_content)
    os.chmod(xstartup_path, 0o755)

    subprocess.run(f"chown -R {USER_NAME}:{USER_NAME} {HOME_DIR}", shell=True)
    print("    ✅ Configuration Complete.")

def create_panic_button():
    print("🚨 [7/9] Creating Panic Button...")

    panic_script_path = "/usr/local/bin/panic-action.sh"
    panic_script = """#!/bin/bash
# Broadcast Message
echo "⚠️ PANIC BUTTON TRIGGERED. DISCONNECTING RUNTIME..." | wall

# 1. Force Unmount
umount -f /content/drive > /dev/null 2>&1

# 2. Kill Tunnels
pkill cloudflared
pkill node

# 3. Wipe Data
rm -rf /content/*
rm -rf /home/colab/*

# 4. Kill Runtime
kill -9 -1
"""
    with open(panic_script_path, "w") as f:
        f.write(panic_script)
    subprocess.run(f"chmod +x {panic_script_path}", shell=True)

    desktop_dir = f"{HOME_DIR}/Desktop"
    os.makedirs(desktop_dir, exist_ok=True)

    desktop_file_path = f"{desktop_dir}/Panic.desktop"
    desktop_content = f"""[Desktop Entry]
Version=1.0
Type=Application
Name=PANIC WIPE
Comment=Instantly wipe data and kill session
Exec=sudo {panic_script_path}
Icon=system-shutdown
Path=
Terminal=false
StartupNotify=false
"""
    with open(desktop_file_path, "w") as f:
        f.write(desktop_content)

    subprocess.run(f"chmod +x {desktop_file_path}", shell=True)
    subprocess.run(f"chown -R {USER_NAME}:{USER_NAME} {HOME_DIR}", shell=True)
    print("    ✅ Panic Button created.")

def start_services():
    print("🚀 [8/9] Starting Services...")

    subprocess.run("vncserver -kill :1", shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    subprocess.run("rm -rf /tmp/.X1-lock /tmp/.X11-unix/X1", shell=True)

    cmd = f"su - {USER_NAME} -c 'vncserver :1 -geometry 1280x720 -depth 16 -localhost no'"
    subprocess.run(cmd, shell=True)

    subprocess.Popen(["/opt/noVNC/utils/novnc_proxy", "--vnc", "localhost:5901", "--listen", "6080"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    print("    ✅ Services Running.")

def start_tunnels():
    print("🔗 [9/9] Generating Public Links...")

    if not os.path.exists(f"{SYSTEM_PATH}/cloudflared"):
        subprocess.run(f"wget -q -O {SYSTEM_PATH}/cloudflared https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64", shell=True)
        subprocess.run(f"chmod +x {SYSTEM_PATH}/cloudflared", shell=True)

    cf_cmd = [f"{SYSTEM_PATH}/cloudflared", "tunnel", "--url", "http://localhost:6080", "--no-autoupdate"]
    cf_proc = subprocess.Popen(cf_cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

    lt_cmd = ["npx", "localtunnel", "--port", "6080"]
    lt_proc = subprocess.Popen(lt_cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

    try:
        public_ip = urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n")
    except:
        public_ip = "Unknown"

    cf_url = "Waiting..."
    start_time = time.time()
    while time.time() - start_time < 20:
        line = cf_proc.stderr.readline()
        if "trycloudflare.com" in line:
            try:
                base_url = "https://" + line.split("https://")[1].split()[0]
                cf_url = f"{base_url}/vnc.html?password={VNC_PASSWORD}&autoconnect=true&resize=scale"
                break
            except:
                pass

    time.sleep(3)
    return cf_url, public_ip, cf_proc, lt_proc

# ==========================================
# MAIN EXECUTION
# ==========================================
def main():
    # 1. Hardware Check
    has_gpu = detect_gpu()
    mode_str = "GPU Mode" if has_gpu else "CPU Mode"

    # 2. Setup
    drive_status = mount_drive()
    install_system(has_gpu)
    create_user()
    setup_novnc()
    configure_vnc_user()
    create_panic_button()
    start_services()

    # 3. Tunnels
    cf_url, public_ip, cf_proc, lt_proc = start_tunnels()

    # 4. Final Stats
    os_ver, ram, storage = get_system_info()

    print("\n" + "="*70)
    print(f"""
    ███╗   ██╗ ██████╗ ██╗   ██╗███╗   ██╗ ██████╗
    ████╗  ██║██╔═══██╗██║   ██║████╗  ██║██╔════╝
    ██╔██╗ ██║██║   ██║██║   ██║██╔██╗ ██║██║
    ██║╚██╗██║██║   ██║╚██╗ ██╔╝██║╚██╗██║██║
    ██║ ╚████║╚██████╔╝ ╚████╔╝ ██║ ╚████║╚██████╗
    ╚═╝  ╚═══╝ ╚═════╝   ╚═══╝  ╚═╝  ╚═══╝ ╚═════╝
    """)
    print("="*70)
    print(f"🖥️  OS:       {os_ver}")
    print(f"🧠  Ram:      {ram}")
    print(f"💾  Storage:  {storage}")
    print(f"⚡  Mode:     {mode_str}")
    print(f"📂  Drive:    {drive_status}")
    print("-" * 70)
    print(f"👤  Username: {USER_NAME}")
    print(f"🔑  Password: {VNC_PASSWORD} (Auto-filled in link)")
    print(f"🛠️  Creator:  Q.SARDOR")
    print("="*70)
    print(f"🔗  LINK 1 (Cloudflare):")
    print(f"    {cf_url}")
    print("-" * 70)
    print(f"🔗  LINK 2 (Backup):")
    print(f"    URL: https://localtunnel.me")
    print(f"    PWD: {public_ip}")
    print("="*70)
    print("⚠️  WARNING: This is a temporary disposable machine.")
    print("    It does NOT keep files after disconnection.")
    print("    Please backup your data to Google Drive immediately.")
    print("="*70 + "\n")

    try:
        while True:
            if cf_proc.poll() is not None:
                print("❌ Cloudflare Tunnel crashed. Restarting...")
                cf_proc = subprocess.Popen([f"{SYSTEM_PATH}/cloudflared", "tunnel", "--url", "http://localhost:6080", "--no-autoupdate"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

            if lt_proc.poll() is not None:
                 lt_proc = subprocess.Popen(["npx", "localtunnel", "--port", "6080"], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

            time.sleep(10)
    except KeyboardInterrupt:
        print("\n🛑 Script stopped by user.")

if __name__ == "__main__":
    main()